In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 1. Carregar o dataset
url = "https://github.com/cassiusf/datasets/raw/refs/heads/main/titanic_data.csv"
df = pd.read_csv(url)

# 1.a. Eliminar variáveis não necessárias
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# 1.b. Elimine observações que contenham dados ausentes (NA)
df = df.dropna()

# 1.c. Aplicar LabelEncoder em 'Embarked' e 'Sex'
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Embarked'] = le.fit_transform(df['Embarked'])

# 1.d. Separe o dataframe resultante em Treino-Teste, utilizando a proporção 75-25%
X = df.drop(columns=['Survived'])
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# 1.e. aplique um modelo SVM utilizando um Kernel linear
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

In [2]:
# 2. Apresente as métricas acurácia, precision, recall e F1-score deste modelo
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nAcurácia:", accuracy_score(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, digits=4))

Matriz de Confusão:
[[89 17]
 [23 49]]

Acurácia: 0.7752808988764045

Relatório de Classificação:
              precision    recall  f1-score   support

           0     0.7946    0.8396    0.8165       106
           1     0.7424    0.6806    0.7101        72

    accuracy                         0.7753       178
   macro avg     0.7685    0.7601    0.7633       178
weighted avg     0.7735    0.7753    0.7735       178



In [3]:
# 3. Execute o mesmo modelo, agora utilizando dois núcleos diferentes, o rbf e o sigmoid.
def treinar_avaliar_A(kernel):
    print(f"\n===== SVM com kernel: {kernel} =====")
    model = SVC(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))

    print("\nAcurácia:", accuracy_score(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, digits=4))


treinar_avaliar_A("rbf")
treinar_avaliar_A("sigmoid")


===== SVM com kernel: rbf =====

Matriz de Confusão:
[[90 16]
 [44 28]]

Acurácia: 0.6629213483146067

Relatório de Classificação:
              precision    recall  f1-score   support

           0     0.6716    0.8491    0.7500       106
           1     0.6364    0.3889    0.4828        72

    accuracy                         0.6629       178
   macro avg     0.6540    0.6190    0.6164       178
weighted avg     0.6574    0.6629    0.6419       178


===== SVM com kernel: sigmoid =====

Matriz de Confusão:
[[65 41]
 [43 29]]

Acurácia: 0.5280898876404494

Relatório de Classificação:
              precision    recall  f1-score   support

           0     0.6019    0.6132    0.6075       106
           1     0.4143    0.4028    0.4085        72

    accuracy                         0.5281       178
   macro avg     0.5081    0.5080    0.5080       178
weighted avg     0.5260    0.5281    0.5270       178



In [4]:
# 4.Compare as três execuções (kernel linear, rbf e sigmoid), apresente um resumo dos resultados relevantes e indique qual o melhor, a partir do experimento
def avaliar_modelo(kernel):
    model = SVC(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    return {
        "Kernel": kernel,
        "Acurácia": acc,
        "Precisão": prec,
        "Recall": rec,
        "F1-score": f1,
        "Matriz Confusão": cm
    }

resultados = [avaliar_modelo(k) for k in ["linear", "rbf", "sigmoid"]]
df_resultados = pd.DataFrame([
    {k: v for k, v in res.items() if k != "Matriz Confusão"} for res in resultados
])

print("===== Comparação de Desempenho =====")
print(df_resultados.to_string(index=False))
print("\n===== Matrizes de Confusão =====")
for res in resultados:
    print(f"\nKernel: {res['Kernel']}")
    print(res["Matriz Confusão"])

# indicar qual foi o melhor...
melhor = max(resultados, key=lambda x: x["F1-score"])
print("\n===== Conclusão =====")
print(f"O melhor kernel foi **{melhor['Kernel']}**, com F1-score = {melhor['F1-score']:.4f}")

===== Comparação de Desempenho =====
 Kernel  Acurácia  Precisão   Recall  F1-score
 linear  0.775281  0.742424 0.680556  0.710145
    rbf  0.662921  0.636364 0.388889  0.482759
sigmoid  0.528090  0.414286 0.402778  0.408451

===== Matrizes de Confusão =====

Kernel: linear
[[89 17]
 [23 49]]

Kernel: rbf
[[90 16]
 [44 28]]

Kernel: sigmoid
[[65 41]
 [43 29]]

===== Conclusão =====
O melhor kernel foi **linear**, com F1-score = 0.7101
